In [2]:
import sqlite3
import requests
import re
from bs4 import BeautifulSoup, Comment
import json
import pandas as pd
import time
import folium
import matplotlib.pyplot as plt
import urllib.request
from selenium import webdriver
import random
#from  brew_api_key  import  key_request, api_key

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

### Checking the Sandbox (r)

In [ ]:
r = requests.get("https://sandbox-api.brewerydb.com/v2/beers/?key=36f6424e1768ab3c31ef2b8a723f4610")

In [ ]:
print(r.status_code)

In [ ]:
print(r.json())

In [ ]:
conn = sqlite3.connect('brewery_db.db') # Creating a new DB
cursor = conn.cursor()                       # cursor allows us to process individual rows

In [ ]:
api_key = '36f6424e1768ab3c31ef2b8a723f4610'
key_request =  'http://api.brewerydb.com/v2/{endpoint}/?key=36f6424e1768ab3c31ef2b8a723f4610'

In [ ]:
headers = {'Content-Type': 'application/json',
           'Authorization': 'Bearer {}'.format(api_key)}

### Checking production (orange)

In [ ]:
orange = requests.get('http://api.brewerydb.com/v2/beers/?key=36f6424e1768ab3c31ef2b8a723f4610')

In [ ]:
print(orange.status_code)

In [ ]:
print(orange.json())

In [ ]:
orange = pd.DataFrame(orange)

In [ ]:
orange.value_counts()

### Checking ingredients (pear)

In [ ]:
pear = requests.get('http://api.brewerydb.com/v2/beer/MgdJep/hops/?key=36f6424e1768ab3c31ef2b8a723f4610')

In [ ]:
print(pear.status_code)

In [ ]:
print(pear.json())

### Webscrapping Beer Advocate for Tables (BS)

In [3]:
url = 'https://www.beeradvocate.com/beer/styles/9/'
response = requests.get(url)

In [4]:
def get_page_contents(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'lxml')

soup = get_page_contents(url)
soup

<!DOCTYPE html>
<html class="Public NoJs uix_javascriptNeedsInit LoggedOut Sidebar Responsive pageIsLtr not_hasTabLinks hasSearch is-sidebarOpen hasRightSidebar is-setWidth navStyle_0 pageStyle_0 hasFlexbox" dir="LTR" id="XenForo" lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<base href="https://www.beeradvocate.com/community/"/>
<script>
			var _b = document.getElementsByTagName('base')[0], _bH = "https://www.beeradvocate.com/community/";
			if (_b && _b.href != _bH) _b.href = _bH;
		</script>
<title>Fruit and Field Beer | BeerAdvocate</title>
<noscript><style>.JsOnly, .jsOnly { display: none !important; }</style></noscript>
<link href="css.php?css=xenforo,form,public&amp;style=6&amp;dir=LTR&amp;d=1616543404" rel="stylesheet"/>
<link href="css.php?css=login_bar,moderator_bar,uix_extendedFooter&amp;style=6&amp;

In [5]:
bs = BeautifulSoup(response.content, 'lxml')
table = bs.table

In [6]:
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

['Style Examples - 1 to 50 (out of 7211) - Ordered by # Reviews']
['\nfirst ← prev |\xa01-50\xa0|\xa051-100\xa0|\xa0101-150\xa0 | next → last']
['\nName\n', '\nBrewery\n', '\nABV\n', '\nRatings\xa0>\n', '\nAvg\n']
['#9', 'Magic Hat Brewing Company', '5.10', '5,915', '3.42', '']
['Samuel Adams Cherry Wheat', 'Boston Beer Company (Samuel Adams)', '5.30', '4,029', '3.19', '']
['Raspberry Tart', 'New Glarus Brewing Company', '4.00', '3,994', '4.41', '']
['Summer Shandy', 'Jacob Leinenkugel Brewing Company', '4.20', '3,841', '3.2', '']
['Rübæus', 'Founders Brewing Company', '5.70', '3,800', '3.91', '']
['Wisconsin Belgian Red', 'New Glarus Brewing Company', '4.00', '3,782', '4.42', '']
['Purple Haze', 'Abita Brewing Co.', '4.20', '3,528', '3.32', '']
['Banana Bread Beer', 'Eagle Brewery', '5.20', '3,235', '3.56', '']
['Serendipity', 'New Glarus Brewing Company', '4.00', '2,994', '4.38', '']
['Hell Or High Watermelon Wheat Beer', '21st Amendment Brewery', '4.90', '2,948', '3.4', '']
['Blushi

In [7]:
df = pd.read_html(str(table))
df = df[0].dropna(axis= 0, thresh = 4)
df

,0,1,2,3,4,5
0,Style Examples - 1 to 50 (out of 7211) - Order...,Style Examples - 1 to 50 (out of 7211) - Order...,Style Examples - 1 to 50 (out of 7211) - Order...,Style Examples - 1 to 50 (out of 7211) - Order...,Style Examples - 1 to 50 (out of 7211) - Order...,NaN
1,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,first ← prev | 1-50 | 51-100 | 101-150 | next ...,NaN
2,Name,Brewery,ABV,Ratings >,Avg,NaN
3,#9,Magic Hat Brewing Company,5.10,5915,3.42,NaN
4,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,4029,3.19,NaN
5,Raspberry Tart,New Glarus Brewing Company,4.00,3994,4.41,NaN
6,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,3841,3.2,NaN
7,Rübæus,Founders Brewing Company,5.70,3800,3.91,NaN
8,Wisconsin Belgian Red,New Glarus Brewing Company,4.00,3782,4.42,NaN
9,Purple Haze,Abita Brewing Co.,4.20,3528,3.32,NaN


In [9]:
#this function will create a df from the table
def beers(url):
    
    response = requests.get(url)
    bs = BeautifulSoup(response.content, 'lxml')
    table = bs.table
    df = pd.read_html(str(table))
    df = df[0].dropna(axis= 0, thresh = 4)

In [8]:
#this function will let us go from page to page
def next_page():
    quarter = driver.find_element_by_tag_name('tbody')
    x = quarter.find_elements_by_tag_name('span')[1].find_elements_by_tag_name('a')[-2]
    x.click()

In [10]:
#this function will iterate and create a table with all of the beers
def total_beers(url):
    
    beers(url)
    for i in x:
        next_page(i)
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'lxml')
        table = bs.table
        df1 = pd.read_html(str(table))
        df1 = df1[0].dropna(axis= 0, thresh = 4)
        result = pd.concat([df, df1], axis = 1)
        time.sleep(random.choice([x/10 for x in range(2,15)]))
        return result

In [11]:
total_beers(url)

NameError: name 'x' is not defined

In [12]:
for i in next_page():
        response = request.get(url)
        bs = BeautifulSoup(response.content, 'lxml')
        table = bs.table
        df1 = pd.read_html(str(table))
        df1 = df[0].dropna(axis= 0, thresh = 4)
        result = pd.concat([df, df1], axis = 1)
        return result

SyntaxError: 'return' outside function (<ipython-input-12-acc59df5d5f9>, line 8)

In [13]:
soup = BeautifulSoup(response.content, 'lxml')
reviews = []
reviews_selector = soup.find_all('div', class_='user_comment')

for review_selector in reviews_selector:
    review_div = review_selector.find('div', class_='user_comment')
    if review_div is None:
        review_div = review_selector.find('div', class_='user_comment')
    review = review_div.find('div', class_='entry').find('p').get_text()
    reviews.append(review)
    

In [14]:
reviews

[]

### Iterating Tables

In [48]:
#This allows us to create an accurate range for the function to iterate through
body = driver.find_element_by_tag_name('tbody')
x = body.find_element_by_tag_name('b').text
pages = int(''.join([i for i in x.split('(')[1] if i.isnumeric()])) // 50

In [ ]:
''' I need to create a list with all the style types and a list with all the styles urls'''

In [51]:
long_list = [('Fruit and Field', 'https://www.beeradvocate.com/beer/styles/9/', pages)]

In [ ]:
''' the funtion below will scrap the beer data from the site'''

In [52]:
data = []
for beer_style, url, pages in long_list:
    driver.get(url)
    for i in tqdm(range(pages)):
        print(i)
        table = driver.find_element_by_tag_name('table')
        tds = [x.find_elements_by_tag_name('td') for x in table.find_elements_by_tag_name('tr')][3:]
        for td in tds:
            data.append([beer_style] + [x.text for x in td] + [td[0].find_element_by_tag_name('a').get_attribute('href')])
            
        quarter = driver.find_element_by_tag_name('tbody')
        x = quarter.find_elements_by_tag_name('span')[1].find_elements_by_tag_name('a')[-2]
        x.click()
        time.sleep(random.choice([x/10 for x in range(8,14)]))

<ipython-input-52-2f495d698de0>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(pages)):


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143



In [161]:
tds = [x.find_elements_by_tag_name('td') for x in table.find_elements_by_tag_name('tr')][3:]

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=89.0.4389.90)


In [57]:
beer_df = pd.DataFrame(data)

In [15]:
beer_df.head(54)

NameError: name 'beer_df' is not defined

In [266]:
beer_df.to_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\beer_df.csv', index = False)

In [243]:
beer_df.isna().sum()

0      0
1      0
2      0
3    144
4    144
5    144
6    144
7    144
dtype: int64

In [78]:
len(beer_df)

7344

In [63]:
beer_tags = BeautifulSoup(response.content, 'html.parser')

In [64]:
tags = [tag.get('href') for tag in beer_tags.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]

In [66]:
tags

['/beer/profile/96/299/',
 '/beer/profile/35/108/',
 '/beer/profile/590/1585/',
 '/beer/profile/710/36010/',
 '/beer/profile/1199/23474/',
 '/beer/profile/590/1577/',
 '/beer/profile/3/7/',
 '/beer/profile/664/5488/',
 '/beer/profile/590/86172/',
 '/beer/profile/735/4202/',
 '/beer/profile/1199/25852/',
 '/beer/profile/22/3523/',
 '/beer/profile/16866/44932/',
 '/beer/profile/35/81388/',
 '/beer/profile/590/20014/',
 '/beer/profile/64/32437/',
 '/beer/profile/590/96831/',
 '/beer/profile/1199/96814/',
 '/beer/profile/1199/48911/',
 '/beer/profile/3/23505/',
 '/beer/profile/35/1903/',
 '/beer/profile/143/67274/',
 '/beer/profile/64/70602/',
 '/beer/profile/64/21466/',
 '/beer/profile/9629/18021/',
 '/beer/profile/273/1971/',
 '/beer/profile/212/38795/',
 '/beer/profile/403/25649/',
 '/beer/profile/105/87280/',
 '/beer/profile/137/345/',
 '/beer/profile/817/68049/',
 '/beer/profile/256/31931/',
 '/beer/profile/17981/73070/',
 '/beer/profile/2372/53146/',
 '/beer/profile/651/1782/',
 '/be

In [200]:
#this is creating a list of the links to the beer profile
i = 0
list_links = []
for i in tqdm(range(0, 300,50)):
    url = f'https://www.beeradvocate.com/beer/styles/9/?sort=revsD&start={i}'
    #if  
    page = requests.get(url)
    beer_tags = BeautifulSoup(response.content, 'html.parser')
    tags = [tag.get('href') for tag in beer_tags.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]
    list_links.append(tags)
    print(f'i is {i}')
#     quarter = driver.find_element_by_tag_name('tbody')
#     x = quarter.find_elements_by_tag_name('span')[1].find_elements_by_tag_name('a')[-2]
#     x.click()
    
    i += 50



<ipython-input-200-4fea937d133b>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, 300,50)):


i is 0
i is 50
i is 100
i is 150
i is 200
i is 250



In [199]:
for tag in beer_tags.find_all(href=re.compile("/beer/styles/[0-9]+/\?sort=revsD")):
        if tag.contents[0] == 'last':
            num = int(tag.get('href').strip(f"{url}?sort=revsD&start"))
            
num

72

In [204]:
new_list_links = []
# j = 3
# for url_end in list_beer_style_urls[0:2]:
#     url = f'https://www.beeradvocate.com{url_end}'
#     print(f'j is {j}')
#     page = requests.get(url)
#     soup = BS(page.content, 'html.parser')

for tag in beer_tags.find_all(href=re.compile("/beer/styles/[0-9]+/\?sort=revsD")):
        if tag.contents[0] == 'last':
            num = int(tag.get('href').strip(f"{url}?sort=revsD&start"))
some_num = 7206 #num+50
i = 0
for i in tqdm(range(0, some_num, 50)):
        print(f'i is {i}')
        url = f'https://www.beeradvocate.com/beer/styles/9/?sort=revsD&start={i}'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        tags = [tag.get('href') for tag in soup.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]
        new_list_links.append(tags)
        i += 50
        
        time.sleep(random.choice([x/10 for x in range(8,14)]))

<ipython-input-204-2e37b2ac55d3>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, some_num, 50)):


i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
i is 1200
i is 1250
i is 1300
i is 1350
i is 1400
i is 1450
i is 1500
i is 1550
i is 1600
i is 1650
i is 1700
i is 1750
i is 1800
i is 1850
i is 1900
i is 1950
i is 2000
i is 2050
i is 2100
i is 2150
i is 2200
i is 2250
i is 2300
i is 2350
i is 2400
i is 2450
i is 2500
i is 2550
i is 2600
i is 2650
i is 2700
i is 2750
i is 2800
i is 2850
i is 2900
i is 2950
i is 3000
i is 3050
i is 3100
i is 3150
i is 3200
i is 3250
i is 3300
i is 3350
i is 3400
i is 3450
i is 3500
i is 3550
i is 3600
i is 3650
i is 3700
i is 3750
i is 3800
i is 3850
i is 3900
i is 3950
i is 4000
i is 4050
i is 4100
i is 4150
i is 4200
i is 4250
i is 4300
i is 4350
i is 4400
i is 4450
i is 4500
i is 4550
i is 4600
i is 4650
i is 4700
i is 4750
i is 4800
i is 4850
i is 4900
i is 4950
i is 5000
i is 5050
i i

In [203]:
new_list_links

[['/beer/profile/96/299/',
  '/beer/profile/35/108/',
  '/beer/profile/590/1585/',
  '/beer/profile/710/36010/',
  '/beer/profile/1199/23474/',
  '/beer/profile/590/1577/',
  '/beer/profile/3/7/',
  '/beer/profile/664/5488/',
  '/beer/profile/590/86172/',
  '/beer/profile/735/4202/',
  '/beer/profile/1199/25852/',
  '/beer/profile/22/3523/',
  '/beer/profile/16866/44932/',
  '/beer/profile/35/81388/',
  '/beer/profile/590/20014/',
  '/beer/profile/64/32437/',
  '/beer/profile/590/96831/',
  '/beer/profile/1199/96814/',
  '/beer/profile/1199/48911/',
  '/beer/profile/3/23505/',
  '/beer/profile/35/1903/',
  '/beer/profile/143/67274/',
  '/beer/profile/64/70602/',
  '/beer/profile/64/21466/',
  '/beer/profile/9629/18021/',
  '/beer/profile/273/1971/',
  '/beer/profile/212/38795/',
  '/beer/profile/403/25649/',
  '/beer/profile/105/87280/',
  '/beer/profile/137/345/',
  '/beer/profile/817/68049/',
  '/beer/profile/256/31931/',
  '/beer/profile/17981/73070/',
  '/beer/profile/2372/53146/',

In [207]:
list_of_beer_profiles = [y for x in new_list_links for y in x]

In [201]:
list_links

[['/beer/profile/96/299/',
  '/beer/profile/35/108/',
  '/beer/profile/590/1585/',
  '/beer/profile/710/36010/',
  '/beer/profile/1199/23474/',
  '/beer/profile/590/1577/',
  '/beer/profile/3/7/',
  '/beer/profile/664/5488/',
  '/beer/profile/590/86172/',
  '/beer/profile/735/4202/',
  '/beer/profile/1199/25852/',
  '/beer/profile/22/3523/',
  '/beer/profile/16866/44932/',
  '/beer/profile/35/81388/',
  '/beer/profile/590/20014/',
  '/beer/profile/64/32437/',
  '/beer/profile/590/96831/',
  '/beer/profile/1199/96814/',
  '/beer/profile/1199/48911/',
  '/beer/profile/3/23505/',
  '/beer/profile/35/1903/',
  '/beer/profile/143/67274/',
  '/beer/profile/64/70602/',
  '/beer/profile/64/21466/',
  '/beer/profile/9629/18021/',
  '/beer/profile/273/1971/',
  '/beer/profile/212/38795/',
  '/beer/profile/403/25649/',
  '/beer/profile/105/87280/',
  '/beer/profile/137/345/',
  '/beer/profile/817/68049/',
  '/beer/profile/256/31931/',
  '/beer/profile/17981/73070/',
  '/beer/profile/2372/53146/',

In [96]:
len(list_links)

7207

In [146]:
first_trial = list_links[0][49:52]

In [125]:
def get_beer_reviews(list_urls):
    list_beers = []
    i = 1
    for url_end in tqdm(list_urls):
        beer = {}
        url = f'https://www.beeradvocate.com{url_end}'
        beer['url'] = url
        print(i)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser') 
        
        listy = soup.find_all(class_ = 'user-comment')
        beer['review'] = []
        for item in listy:
            beer['review'].append(item.get_text())
            
#         titlebar = soup.find(class_='titleBar')
#         for span in titlebar('span'):
#             span.decompose()

#         beer['name'] = titlebar.get_text().strip()
        
        list_beers.append(beer)
        
        time.sleep(random.choice([x/10 for x in range(8,14)]))
        
        i += 1
    return list_beers

In [147]:
demo_df = get_beer_reviews(first_trial)

<ipython-input-125-dd199f7b9ad1>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url_end in tqdm(list_urls):


1



In [148]:
pd.DataFrame(demo_df)

,url,review
0,https://www.beeradvocate.com/beer/profile/113/...,[DrBeergood from Ohio4/5 rDev +3.4%look: 4 | ...


In [129]:
list_of_beer_profiles = [y for x in list_links for y in x]

In [208]:
list_of_beer_profiles[:200]

['/beer/profile/96/299/',
 '/beer/profile/35/108/',
 '/beer/profile/590/1585/',
 '/beer/profile/710/36010/',
 '/beer/profile/1199/23474/',
 '/beer/profile/590/1577/',
 '/beer/profile/3/7/',
 '/beer/profile/664/5488/',
 '/beer/profile/590/86172/',
 '/beer/profile/735/4202/',
 '/beer/profile/1199/25852/',
 '/beer/profile/22/3523/',
 '/beer/profile/16866/44932/',
 '/beer/profile/35/81388/',
 '/beer/profile/590/20014/',
 '/beer/profile/64/32437/',
 '/beer/profile/590/96831/',
 '/beer/profile/1199/96814/',
 '/beer/profile/1199/48911/',
 '/beer/profile/3/23505/',
 '/beer/profile/35/1903/',
 '/beer/profile/143/67274/',
 '/beer/profile/64/70602/',
 '/beer/profile/64/21466/',
 '/beer/profile/9629/18021/',
 '/beer/profile/273/1971/',
 '/beer/profile/212/38795/',
 '/beer/profile/403/25649/',
 '/beer/profile/105/87280/',
 '/beer/profile/137/345/',
 '/beer/profile/817/68049/',
 '/beer/profile/256/31931/',
 '/beer/profile/17981/73070/',
 '/beer/profile/2372/53146/',
 '/beer/profile/651/1782/',
 '/be

In [209]:
len(list_of_beer_profiles)

7206

In [210]:
reviews = get_beer_reviews(list_of_beer_profiles)

<ipython-input-125-dd199f7b9ad1>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url_end in tqdm(list_urls):


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


In [220]:
beers_1000 = pd.DataFrame(reviews)
beers_1000

,url,review
0,https://www.beeradvocate.com/beer/profile/96/299/,[Elemental19xx from Illinois3.6/5 rDev +5.3%l...
1,https://www.beeradvocate.com/beer/profile/35/108/,[BergBeer from Virginia1.15/5 rDev -63.9%look...
2,https://www.beeradvocate.com/beer/profile/590/...,[TedHead from Illinois4.85/5 rDev +10%look: 4...
3,https://www.beeradvocate.com/beer/profile/710/...,[AlCaponeJunior from Texas2.79/5 rDev -12.8%l...
4,https://www.beeradvocate.com/beer/profile/1199...,[TomQBigBeerGuy from Michigan3.88/5 rDev -0.8...
...,...,...
7201,https://www.beeradvocate.com/beer/profile/5695...,[]
7202,https://www.beeradvocate.com/beer/profile/4786...,[]
7203,https://www.beeradvocate.com/beer/profile/5716...,[]
7204,https://www.beeradvocate.com/beer/profile/4454...,[]


In [235]:
beers_1000['review'][7201]

[]

In [236]:
beer_reviews = beers_1000.copy()

In [240]:
beer_reviews.loc[~beer_reviews.review.astype(bool),'review']=np.nan

In [242]:
beer_reviews.isna().sum()

url          0
review    1658
dtype: int64

In [271]:
beer_reviews.to_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\reviews.csv', index = False)

In [273]:
df = beer_df.merge(beer_reviews, on='url', how='left')
df.to_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\beer_w_reviews.csv', index = False)

In [274]:
df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,drop_me,url,review
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,,https://www.beeradvocate.com/beer/profile/96/299/,[Elemental19xx from Illinois3.6/5 rDev +5.3%l...
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,,https://www.beeradvocate.com/beer/profile/35/108/,[BergBeer from Virginia1.15/5 rDev -63.9%look...
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,,https://www.beeradvocate.com/beer/profile/590/...,[TedHead from Illinois4.85/5 rDev +10%look: 4...
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.2,,https://www.beeradvocate.com/beer/profile/710/...,[AlCaponeJunior from Texas2.79/5 rDev -12.8%l...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,,https://www.beeradvocate.com/beer/profile/1199...,[TomQBigBeerGuy from Michigan3.88/5 rDev -0.8...
...,...,...,...,...,...,...,...,...,...
7339,Fruit and Field,Radler,Occy's Brewery,4.80,0,0,,https://www.beeradvocate.com/beer/profile/2128...,NaN
7340,Fruit and Field,Alan-A-Dale Apple Ale,Robin Hood Brewing Co. / Home D Pizzeria,5.00,0,0,,https://www.beeradvocate.com/beer/profile/4182...,NaN
7341,Fruit and Field,Tussle In the Hay,Back Unturned Brewing Company,7.90,0,0,,https://www.beeradvocate.com/beer/profile/5844...,NaN
7342,Fruit and Field,Strawberry Chocolate Cheesecake Sour,Barn Town Brewing,5.90,0,0,,https://www.beeradvocate.com/beer/profile/4760...,NaN


In [270]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,drop_me,url
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,,https://www.beeradvocate.com/beer/profile/96/299/
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,,https://www.beeradvocate.com/beer/profile/35/108/
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,,https://www.beeradvocate.com/beer/profile/590/...
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.2,,https://www.beeradvocate.com/beer/profile/710/...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,,https://www.beeradvocate.com/beer/profile/1199...
...,...,...,...,...,...,...,...,...
7339,Fruit and Field,Radler,Occy's Brewery,4.80,0,0,,https://www.beeradvocate.com/beer/profile/2128...
7340,Fruit and Field,Alan-A-Dale Apple Ale,Robin Hood Brewing Co. / Home D Pizzeria,5.00,0,0,,https://www.beeradvocate.com/beer/profile/4182...
7341,Fruit and Field,Tussle In the Hay,Back Unturned Brewing Company,7.90,0,0,,https://www.beeradvocate.com/beer/profile/5844...
7342,Fruit and Field,Strawberry Chocolate Cheesecake Sour,Barn Town Brewing,5.90,0,0,,https://www.beeradvocate.com/beer/profile/4760...


In [256]:
#beer_df.rename(columns={"0": "beer_style", "1": "beer_name", '2': 'brewery_name','3': 'abv', '4': 'no_ratings','5': 'avg_ratings','6': 'dropme','7': 'url'}, inplace = True)
beer_df.columns = ['beer_style', 'beer_name', 'brewery_name', 'abv', 'no_ratings', 'avg_rating', 'drop_me', 'url']
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,drop_me,url
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,,https://www.beeradvocate.com/beer/profile/96/299/
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,,https://www.beeradvocate.com/beer/profile/35/108/
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,,https://www.beeradvocate.com/beer/profile/590/...
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.2,,https://www.beeradvocate.com/beer/profile/710/...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,,https://www.beeradvocate.com/beer/profile/1199...
...,...,...,...,...,...,...,...,...
7339,Fruit and Field,Radler,Occy's Brewery,4.80,0,0,,https://www.beeradvocate.com/beer/profile/2128...
7340,Fruit and Field,Alan-A-Dale Apple Ale,Robin Hood Brewing Co. / Home D Pizzeria,5.00,0,0,,https://www.beeradvocate.com/beer/profile/4182...
7341,Fruit and Field,Tussle In the Hay,Back Unturned Brewing Company,7.90,0,0,,https://www.beeradvocate.com/beer/profile/5844...
7342,Fruit and Field,Strawberry Chocolate Cheesecake Sour,Barn Town Brewing,5.90,0,0,,https://www.beeradvocate.com/beer/profile/4760...


In [257]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,drop_me,url
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,,https://www.beeradvocate.com/beer/profile/96/299/
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,,https://www.beeradvocate.com/beer/profile/35/108/
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,,https://www.beeradvocate.com/beer/profile/590/...
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.2,,https://www.beeradvocate.com/beer/profile/710/...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,,https://www.beeradvocate.com/beer/profile/1199...
...,...,...,...,...,...,...,...,...
7339,Fruit and Field,Radler,Occy's Brewery,4.80,0,0,,https://www.beeradvocate.com/beer/profile/2128...
7340,Fruit and Field,Alan-A-Dale Apple Ale,Robin Hood Brewing Co. / Home D Pizzeria,5.00,0,0,,https://www.beeradvocate.com/beer/profile/4182...
7341,Fruit and Field,Tussle In the Hay,Back Unturned Brewing Company,7.90,0,0,,https://www.beeradvocate.com/beer/profile/5844...
7342,Fruit and Field,Strawberry Chocolate Cheesecake Sour,Barn Town Brewing,5.90,0,0,,https://www.beeradvocate.com/beer/profile/4760...


In [267]:
df = pd.read_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\beer_df.csv')
df.head(52)

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,drop_me,url
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,NaN,https://www.beeradvocate.com/beer/profile/96/299/
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,NaN,https://www.beeradvocate.com/beer/profile/35/108/
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,NaN,https://www.beeradvocate.com/beer/profile/590/...
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.20,NaN,https://www.beeradvocate.com/beer/profile/710/...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,NaN,https://www.beeradvocate.com/beer/profile/1199...
5,Fruit and Field,Wisconsin Belgian Red,New Glarus Brewing Company,4.00,"3,779",4.42,NaN,https://www.beeradvocate.com/beer/profile/590/...
6,Fruit and Field,Purple Haze,Abita Brewing Co.,4.20,"3,527",3.32,NaN,https://www.beeradvocate.com/beer/profile/3/7/
7,Fruit and Field,Banana Bread Beer,Eagle Brewery,5.20,"3,235",3.56,NaN,https://www.beeradvocate.com/beer/profile/664/...
8,Fruit and Field,Serendipity,New Glarus Brewing Company,4.00,"2,993",4.38,NaN,https://www.beeradvocate.com/beer/profile/590/...
9,Fruit and Field,Hell Or High Watermelon Wheat Beer,21st Amendment Brewery,4.90,"2,948",3.40,NaN,https://www.beeradvocate.com/beer/profile/735/...


In [ ]:
def reviews(url):
    
    tag_name = driver.find_element_by_xpath('//*[@id="ba-content"]/table/tbody/tr[4]/td[1]/a')
    for i in tag_name:
        i.click()
        for y in blah:
            ba_content = driver.find_element_by_xpath('//*[@id="ba-content"]')
            
        

In [ ]:
tag_name = driver.find_element_by_xpath('//*[@id="ba-content"]/table/tbody/tr[4]/td[1]/a').text
tag_name

In [ ]:
ba_content = driver.find_element_by_xpath('//*[@id="ba-content"]').text
ba_content

In [ ]:
u_comment = driver.find_elements_by_class_name('user-comment')
review3 = []
for item in u_comment:
    review3.append(item.text)

In [ ]:
tag_name = driver.find_element_by_xpath('//*[@id="ba-content"]/table/tbody/tr[4]/td[1]/a/b')

In [ ]:
tag_name.text

In [ ]:
beers('https://www.beeradvocate.com/beer/styles/9/')

In [ ]:
name_brew_rating = soup.find('td', class_='hr_bottom_light')

In [ ]:
name_brew_rating

In [ ]:
name = name_brew_rating.a
name

In [ ]:
brew = name_brew_rating.tr
brew

In [ ]:
brew

In [ ]:
for name_brew_rating in tqdm(soup.find_all('td')):
    print(name_brew_rating.get_text())

In [ ]:
for name in soup.find_all('b'):
    print(name.get_text())

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.findAll('a')

In [ ]:
one_tag = soup.findAll('a')[189]
link = one_tag['href']

In [ ]:
link

In [ ]:
download_url = 'https://www.beeradvocate.com/beer/profile/96/299/' + link
urllib.request.urlretrieve(download_url)

In [ ]:
download_url

In [ ]:
# To download the whole data set, let's do a for loop through all a tags
line_count = 1 #variable to track what line you are on
for one_a_tag in soup.findAll('a'):  #'a' tags are for links
    if line_count >= 36: #code for text files starts at line 36
        link = one_a_tag['href']
        download_url = 'http://web.mta.info/developers/'+ link
        urllib.request.urlretrieve(download_url,'./'+link[link.find('/turnstile_')+1:]) 
        time.sleep(1) #pause the code for a sec
    #add 1 for next line
    line_count +=1

### Selenium 

In [ ]:
#testing that Selenium works
from selenium import webdriver
driver = webdriver.Chrome()
driver.get('https://www.nasa.gov')
headlines = driver.find_elements_by_class_name("headline")
for headline in headlines:
    print(headline.text.strip())
driver.close()

In [43]:
#estabishing the webdriver for chrome
driver = webdriver.Chrome()

In [44]:
#putting the driver on the website
driver.get('https://www.beeradvocate.com/beer/styles/9/')

In [ ]:
#clicking into a beer
driver.find_element_by_xpath('//*[@id="ba-content"]/table/tbody/tr[4]/td[1]/a').click()

In [ ]:
#pulling out the comments
comment = driver.find_element_by_xpath('//*[@id="rating_fullview_content_2"]').text

In [ ]:
comment

In [ ]:
#this will allow us to click next on the page
quarter = driver.find_element_by_tag_name('tbody')
x = quarter.find_elements_by_tag_name('span')[1].find_elements_by_tag_name('a')[-2]
x.click()

In [ ]:
#this function will let us go from page to page
def next_page():
    quarter = driver.find_element_by_tag_name('tbody')
    x = quarter.find_elements_by_tag_name('span')[1].find_elements_by_tag_name('a')[-2]
    x.click()

In [ ]:
next_page()

### Testing pulling reviews

In [172]:
name = driver.find_element_by_xpath('//*[@id="ba-content"]/table/tbody/tr[5]/td[1]/a/b').text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ba-content"]/table/tbody/tr[5]/td[1]/a/b"}
  (Session info: chrome=89.0.4389.90)


In [173]:
name

'Samuel Adams Cherry Wheat'

In [174]:
# getting reviews of our beers
review1 = driver.find_element_by_xpath('//*[@id="rating_fullview"]').text

In [175]:
review1

'Elemental19xx from Illinois\n\n3.6/5  rDev +5.3%\nlook: 3 | smell: 3.5 | taste: 3.75 | feel: 3.75 | overall: 3.5\n\nBeer pours a little cloudy with a little head, and a few small particulates. Smells more like a fruity ale than an IPA (which I like). tastes pretty good in my opinion, but not as hoppy as a pale ale( perhaps that\'s why it says not quite pale ale on the front). It\'s like your average run of the mill ale, but just a bit more fruity and a light hop flavor towards the end. The carbonation is mild, not smooth like nitro brews, but relatively soft. Overall I could drink em all day and not be sick of the taste, but it would be my first choice if I was looking to only drink a beer or two after dinner. That being said if I see them in the custom six pack fridge I would be okay with adding one of these.\n\nFeb 28, 2021\nkorfao from North Carolina\n\n4.06/5  rDev +18.7%\nlook: 4.25 | smell: 3.75 | taste: 4.25 | feel: 4 | overall: 4\n\nA definite rich color and apricot/fruit smel

In [176]:
#list of reviews
review2 = driver.find_elements_by_xpath('//*[@id="rating_fullview_content_2"]')

In [177]:
for item in review2:
    print(item.text)

Elemental19xx from Illinois

3.6/5  rDev +5.3%
look: 3 | smell: 3.5 | taste: 3.75 | feel: 3.75 | overall: 3.5

Beer pours a little cloudy with a little head, and a few small particulates. Smells more like a fruity ale than an IPA (which I like). tastes pretty good in my opinion, but not as hoppy as a pale ale( perhaps that's why it says not quite pale ale on the front). It's like your average run of the mill ale, but just a bit more fruity and a light hop flavor towards the end. The carbonation is mild, not smooth like nitro brews, but relatively soft. Overall I could drink em all day and not be sick of the taste, but it would be my first choice if I was looking to only drink a beer or two after dinner. That being said if I see them in the custom six pack fridge I would be okay with adding one of these.

Feb 28, 2021
korfao from North Carolina

4.06/5  rDev +18.7%
look: 4.25 | smell: 3.75 | taste: 4.25 | feel: 4 | overall: 4

A definite rich color and apricot/fruit smell. It claims it'

Jan 05, 2019
Tweezerjam from Pennsylvania

4.64/5  rDev +35.7%
look: 4 | smell: 5 | taste: 4.5 | feel: 4 | overall: 5

One of my all time favorites. Its got a hint of apricot sweetness, but is perfectly balanced with mild hops. I could drink this all day (indeed I have). Its a must try for everyone. Enjoy!

Dec 24, 2018
JimboBrews54 from Michigan

4/5  rDev +17%
look: 4 | smell: 4 | taste: 4 | feel: 4 | overall: 4

Love potion #9 comes to mind. One of the first "craft" beers I have ever enjoyed, it is a blend of both sweet and lager. This one does not over power the senses in any way. If you are looking for an introduction in the fruit and field category, this is a good beer to do that with.

Oct 19, 2018
Nass from Florida

2.7/5  rDev -21.1%
look: 3.5 | smell: 2.25 | taste: 2.75 | feel: 3.5 | overall: 2.5

There are some light fruit notes & it is not too hoppy like an IPA. It's lacking in taste & the body is quite thin. There is nothing really that stands out from this beer but it's s

In [178]:
#creating a list of reviews a different way
u_comment = driver.find_elements_by_class_name('user-comment')
review3 = []
for item in u_comment:
    review3.append(item.text)
review3


["Elemental19xx from Illinois\n\n3.6/5  rDev +5.3%\nlook: 3 | smell: 3.5 | taste: 3.75 | feel: 3.75 | overall: 3.5\n\nBeer pours a little cloudy with a little head, and a few small particulates. Smells more like a fruity ale than an IPA (which I like). tastes pretty good in my opinion, but not as hoppy as a pale ale( perhaps that's why it says not quite pale ale on the front). It's like your average run of the mill ale, but just a bit more fruity and a light hop flavor towards the end. The carbonation is mild, not smooth like nitro brews, but relatively soft. Overall I could drink em all day and not be sick of the taste, but it would be my first choice if I was looking to only drink a beer or two after dinner. That being said if I see them in the custom six pack fridge I would be okay with adding one of these.\n\nFeb 28, 2021",
 "korfao from North Carolina\n\n4.06/5  rDev +18.7%\nlook: 4.25 | smell: 3.75 | taste: 4.25 | feel: 4 | overall: 4\n\nA definite rich color and apricot/fruit sm

### Setting up SQL tables

In [ ]:
TABLES = {}
TABLES['beer_info'] = (
    """CREATE TABLE IF NOT EXISTS business (
        id VARCHAR(30) NOT NULL,
        name VARCHAR,
        nameDisplay VARCHAR,
        description TEXT,
        isRetired TEXT,
        abv DECIMAL,
        ibu DECIMAL,
        originalGravity VARCHAR,
        isOrganic VARCHAR,
        styleId TEXT NOT NULL,
        style VARCHAR,
        PRIMARY KEY (id)
        );
    """)
TABLES['ratings'] = (
    """CREATE TABLE IF NOT EXISTS ratings (
        ratings VARCHAR(30) PRIMARY KEY,
        buisness_id VARCHAR(30) NOT NULL,
        rating DECIMAL,
        text TEXT,
        date_created TEXT,
        FOREIGN KEY (business_id)
            REFERENCES business (business_id)
        );
    """)

### Checking Brewers Friend API (apple)

I need to pay for the access to their premium setup that includes ingredients

https://www.brewersfriend.com/homebrew/profile/membership

How to search for recipes (they are all home brews)

https://docs.brewersfriend.com/api/recipes#search-all-my-recipes

In [ ]:
apple = requests.get(

In [ ]:
curl -H "X-API-Key: 535ad06757dd29bd83be1e635ec10530407a8XAVTX" \
https://api.brewersfriend.com/v1/recipes

In [ ]:
def get_account_info():

    api_url = '{0}account'.format(api_url_base)

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

In [ ]:
/:beerId/ingredients